In [1]:
import os
import torch, torchvision

os.chdir("/home/kishen/documents/python_projects/stanford_dogs/")
from src.data import ImageDataset

In [2]:
#read in image data
path = "/home/kishen/documents/python_projects/stanford_dogs/data/Images"

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(p=.55),
    torchvision.transforms.Normalize(mean= [0.4760, 0.4517, 0.3907], 
                                     std= [0.2362, 0.2314, 0.2307])

])

dataset = ImageDataset(root_directory=path, transform=transform)

In [3]:
#split data
train_size, val_size = int(dataset.__len__() * .8) , int(dataset.__len__() * .2)

ran_gen= torch.Generator().manual_seed(24)

train_data, val_data = torch.utils.data.random_split(dataset, 
                                                    [train_size, val_size],
                                                     generator= ran_gen)




In [4]:
#create dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [5]:
# #calculcate train data stats
# from src.modeling import calculate_channel_means, calculate_channel_sds

# display(calculate_channel_means(train_data, 'image'))
# #tensor([0.4760, 0.4517, 0.3907])

# display(calculate_channel_sds(train_data, 'image'))
# #tensor([0.2362, 0.2314, 0.2307])


## Model Evaluation
We will evaluate performances using EfficientNet, Resnet18, and VGG16

In [8]:
#init models, update FC layers, and move models to gpu
from torchvision import models

device = "cuda" if torch.cuda.is_available() else "cpu"
print("available device: " + device)

efficientNet = models.efficientnet_b0()

#resnet18
resnet18 = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18.fc.in_features

resnet18.fc = torch.nn.Linear(num_ftrs, 120)
resnet18.to(device)


vgg16 = models.vgg16()

# vgg16.classifier[-1] = torch.nn.Linear(in_features= 4096,
#                                        out_features = 120)
# vgg16.to(device)

available device: cuda


In [ ]:
#init criterion and optimizer
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()

### EfficientNet Model Performance

In [ ]:
efficientNet = models.efficientnet_b0()

### Resnet Model Performance

In [ ]:
resnet18 = models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18.fc.in_features

resnet18.fc = torch.nn.Linear(num_ftrs, 120)
resnet18.to(device)
print("resnet18 is on the following device: ", device)


resnet18 is on the following device:  cuda


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer_resnet18 = optim.SGD(resnet18.parameters(), 
                          lr = .001, 
                          momentum= .9)

In [ ]:
from src.modeling import train_model

resnet18_dict = {}

resnet18_dict = train_model(model=resnet18, 
                         dataloader = train_dataloader, 
                         epochs = 5,
                         optimizer = optimizer_resnet18,
                         criterion = criterion, 
                         device = device)

/home/kishen/.virtualenvs/stanford_dogs/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Statistics:
epoch loss:  3.631
epoch acc: 28.15%

Epoch 2 Statistics:
epoch loss:  1.896
epoch acc: 63.62%



### VGG16 Performance

In [7]:
vgg16 = models.vgg16()

# vgg16.classifier[-1] = torch.nn.Linear(in_features= 4096,
#                                        out_features = 120)
# vgg16.to(device)


#init criterion and optimizer
import torch.optim as optim

# criterion = torch.nn.CrossEntropyLoss()
# optimizer_vgg = optim.SGD(vgg16.parameters(), 
#                           lr = .001, 
#                           momentum= .9)


In [8]:
# from src.modeling import train_model

# vgg16_dict = {}

# vgg16_dict = train_model(model=vgg16, 
#                          dataloader = train_dataloader, 
#                          epochs = 1,
#                          optimizer = optimizer_vgg,
#                          criterion = criterion, 
#                          device = device)
